In [ ]:
!wget https://pythonprogramming.net/static/downloads/machine-learning-data/pos.txt
!wget https://pythonprogramming.net/static/downloads/machine-learning-data/neg.txt

In [ ]:
import nltk
nltk.download('all')

In [43]:
import tensorflow as tf
import numpy as np
import random
import nltk
from nltk.tokenize import word_tokenize
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
hm_lines = 100000

def create_lexicon(pos, neg):
    """
    Create a lexicon from positive and negative files.
    """
    """" Return word and their count """
    lexicon = []
    for file in [pos, neg]:
        with open(file, 'r') as f:
            contents = f.readlines()
            for l in contents[:hm_lines]:
                all_words = word_tokenize(l)
                lexicon.extend(all_words)
    lexicon = [lemmatizer.lemmatize(i.lower()) for i in lexicon]
    w_counts = Counter(lexicon)
    lexicon = [w for w, c in w_counts.items() if 50 < c < 1000]
    print(len(lexicon))
    return lexicon

def sample_handling(sample, lexicon, classification):
    """ This function will contvert words in to vectors
        featureset = [
            [ [1 2 4 1], [0,1] ], <- negative sample
            [ [0 1 0 1 1 0], [1,0] ] <- positive sample
        ]
    """
    """
    Handle sample data and create features.
    """
    featureset = []
    with open(sample, 'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            current_words = word_tokenize(l.lower())
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon), dtype=int)
            for word in current_words:
                if word in lexicon:
                    index_value = lexicon.index(word)
                    features[index_value] += 1
            featureset.append([features, classification])
    return featureset

def create_feature_sets_and_labels(pos, neg, test_size=0.1):
    """
    Create feature sets and labels.
    """
    lexicon = create_lexicon(pos, neg)
    features = sample_handling(pos, lexicon, [1, 0]) + sample_handling(neg, lexicon, [0, 1])
    random.shuffle(features)
    features = np.array(features, dtype=object)

    testing_size = int(test_size * len(features))

    train_x, train_y = features[:-testing_size, 0], features[:-testing_size, 1]
    test_x, test_y = features[-testing_size:, 0], features[-testing_size:, 1]

    return train_x, train_y, test_x, test_y

In [44]:
train_x,train_y,test_x,test_y = create_feature_sets_and_labels('pos.txt','neg.txt')


423
